# GPU Accelerated Computing - Assignment 1
Muhammad Meesum Ali Qazalbash (mq06861)

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

# Question 1
Write CUDA code to initialize a random array of 1000000 elements.

In [ ]:
%%cu
#include <cuda.h>
#include <cuda_runtime.h>
#include <curand.h>
#include <curand_kernel.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define N 1000000

inline cudaError_t checkCudaErr(cudaError_t err, const char *msg) {
    if (err != cudaSuccess) {
        fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg,
                cudaGetErrorString(err));
    }
    return err;
}

__global__ void init_rand(curandState *state, unsigned long seed) {
    int id = threadIdx.x + blockIdx.x * blockDim.x;
    curand_init(seed, id, 0, &state[id]);
}

__global__ void generate_rand(curandState *state, float *rand) {
    int id   = threadIdx.x + blockIdx.x * blockDim.x;
    rand[id] = curand_uniform(&state[id]);
}

int main() {
    float       *rande, *d_rand, cpu_time_used, gpu_time_used, memcpy_time_used;
    int          size = N * sizeof(float), state_size = N * sizeof(curandState);
    curandState *state, *d_state;
    clock_t      start, end;
    cudaEvent_t  start_gpu, stop_gpu;

    rande = (float *)malloc(size);
    state = (curandState *)malloc(state_size);

    checkCudaErr(cudaMalloc((void **)&d_rand, size), "cudaMalloc");
    checkCudaErr(cudaMalloc((void **)&d_state, state_size), "cudaMalloc");

    start = clock();
    for (int i = 0; i < N; i++) rande[i] = (float)rand() / (float)RAND_MAX;
    end           = clock();
    cpu_time_used = ((float)(end - start)) / CLOCKS_PER_SEC;
    printf("CPU time used:    %10f seconds to generate random numbers on CPU \n",
           cpu_time_used);
            
    checkCudaErr(cudaEventCreate(&start_gpu), "cudaEventCreate");
    checkCudaErr(cudaEventCreate(&stop_gpu), "cudaEventCreate");
    checkCudaErr(cudaEventRecord(start_gpu, 0), "cudaEventRecord");

    init_rand<<<N / 256, 256>>>(d_state, time(NULL));
    generate_rand<<<N / 256, 256>>>(d_state, d_rand);

    checkCudaErr(cudaEventRecord(stop_gpu, 0), "cudaEventRecord");
    checkCudaErr(cudaEventSynchronize(stop_gpu), "cudaEventSynchronize");
    checkCudaErr(cudaEventElapsedTime(&gpu_time_used, start_gpu, stop_gpu), "cudaEventElapsedTime");

    printf("GPU time used:    %10f seconds to generate random numbers on GPU \n", gpu_time_used / 1000.0f);
            
    start = clock();

    checkCudaErr(cudaMemcpy(rande, d_rand, size, cudaMemcpyDeviceToHost),
                 "cudaMemcpy");
     end = clock();
     memcpy_time_used = ((float)(end - start)) / CLOCKS_PER_SEC;
     printf("MEMCPY time used: %10f seconds to copy random numbers from GPU to CPU\n", memcpy_time_used / 1000.0f);

    checkCudaErr(cudaFree(d_rand), "cudaFree");
    checkCudaErr(cudaFree(d_state), "cudaFree");

    free(rande);
    free(state);


    return 0;
}

# Question 2
Write CUDA code to calculate the sum of 1000 elements array and output the sum. Calculate the total time for doing the calculation on the CPU as well as on the GPU. Use an appropriate execution configuration when you launch your kernel. Share the following information from your solution.
## Explaination
The parallel logic for reduction is used to calculate the sum of the array. The logic is as follows:
1. The array is divided into blocks.
2. Each block is then reduced to a single element using the parallel logic.
3. The final sum is calculated by adding the reduced elements of each block.
## Reference
[GitHub reference](https://github.com/CoffeeBeforeArch/cuda_programming/blob/master/03_sum_reduction/bank_conflicts/sumReduction.cu)
## Plots
They can be found by running the code in the one after this cell.

In [ ]:
%%cu
#include <cuda.h>
#include <cuda_runtime.h>
#include <curand.h>
#include <curand_kernel.h>
#include <stdio.h>
#include <time.h>

#define SHARED_MEMORY_SIZE 250

/**
 * @brief Checks for CUDA errors and prints the error message
 *
 * @param err
 * @param msg
 * @return cudaError_t
 */
inline cudaError_t checkCudaErr(cudaError_t err, const char *msg) {
    if (err != cudaSuccess) {
        fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg,
                cudaGetErrorString(err));
    }
    return err;
}

/**
 * @brief Calculates the sum of an array using a reduction algorithm
 *
 * @param vector The input vector
 * @param reduced_vector The output vector
 */
__global__ void sumThroughReduction(const int *vector, int *reduced_vector) {
    __shared__ int partial_sum[SHARED_MEMORY_SIZE];  // Shared memory
    int id = blockIdx.x * blockDim.x + threadIdx.x;  // Global thread ID
    partial_sum[threadIdx.x] = vector[id];  // Load data into shared memory

    __syncthreads();  // Wait for all threads to load data into shared memory
                      // before starting the reduction

    // Reduction in shared memory (sequential addressing)
    int index;
    for (int s = 1; s < blockDim.x; s *= 2) {
        // Each thread does work unless the index goes off the block (s is the
        // stride)
        index = 2 * s * threadIdx.x;  // Index of the element to be added

        // Add the element at index to the element at index + s
        if (index < blockDim.x) partial_sum[index] += partial_sum[index + s];

        __syncthreads();  // Wait for all threads to finish before starting the
                          // next iteration
    }

    // Write the result for this block to the output vector at the correct index
    if (threadIdx.x == 0) reduced_vector[blockIdx.x] = partial_sum[0];
}

/**
 * @brief The main function of the program
 *
 * @return int
 */
int main(void) {
    float       cpu_time_used, gpu_time_used;
    clock_t     start, end;
    cudaEvent_t start_gpu, stop_gpu;

    int ARRAY_SIZES[3] = {10000, 100000, 1000000};

    FILE *fp = fopen("output.csv", "w");
    fprintf(fp, "Dataset size,Grid size,Thread size,CPU time,GPU time\n");

    for (int ARRAY_SIZE : ARRAY_SIZES) {
        // Size of the array in bytes
        int size = ARRAY_SIZE * sizeof(int);

        // Host data (initial and final) (CPU)
        int *initial_host_data = (int *)malloc(size);
        int *final_host_data   = (int *)malloc(size);

        int i = 0;

        // Initialize the host data with some values (1,2,3,...) and set the
        // final host data to 0
        for (; i < ARRAY_SIZE; i++) {
            initial_host_data[i] = i + 1;
            final_host_data[i]   = 0;
        }

        start   = clock();
        int sum = 0;
        // Calculate the sum on the host (CPU)
        for (i = 0; i < ARRAY_SIZE;) sum += initial_host_data[i++];
        end           = clock();
        cpu_time_used = ((float)(end - start)) / CLOCKS_PER_SEC;
        printf("CPU time used: %f seconds for sum = %d\n", cpu_time_used, sum);

        int GRIDS[] = {ARRAY_SIZE / 250, ARRAY_SIZE / 500, ARRAY_SIZE / 1000};
        int THREADS = 250;

        // Device data (initial and final) (GPU)
        int *initial_device_data, *final_device_data;
        checkCudaErr(cudaMalloc((void **)&initial_device_data, size),
                     "cudaMalloc");
        checkCudaErr(cudaMalloc((void **)&final_device_data, size),
                     "cudaMalloc");

        for (int i = 0; i < 3; i++) {
            // Copy data from host to device (CPU -> GPU)
            checkCudaErr(cudaMemcpy(initial_device_data, initial_host_data,
                                    size, cudaMemcpyHostToDevice),
                         "cudaMemcpy");
            checkCudaErr(cudaMemcpy(final_device_data, final_host_data, size,
                                    cudaMemcpyHostToDevice),
                         "cudaMemcpy");

            // Create events to measure the time taken by the GPU
            checkCudaErr(cudaEventCreate(&start_gpu), "cudaEventCreate");
            checkCudaErr(cudaEventCreate(&stop_gpu), "cudaEventCreate");
            checkCudaErr(cudaEventRecord(start_gpu, 0), "cudaEventRecord");

            // Launch the kernel
            sumThroughReduction<<<GRIDS[i], THREADS>>>(initial_device_data,
                                                       final_device_data);

            // Launch the kernel again to reduce the final
            // data to a single value (the sum)
            sumThroughReduction<<<1, THREADS>>>(final_device_data,
                                                final_device_data);

            // Copy data from device to host (GPU -> CPU)
            checkCudaErr(cudaMemcpy(final_host_data, final_device_data, size,
                                    cudaMemcpyDeviceToHost),
                         "cudaMemcpy");

            checkCudaErr(cudaEventRecord(stop_gpu, 0), "cudaEventRecord");
            checkCudaErr(cudaEventSynchronize(stop_gpu),
                         "cudaEventSynchronize");
            checkCudaErr(
                cudaEventElapsedTime(&gpu_time_used, start_gpu, stop_gpu),
                "cudaEventElapsedTime");

            // Print the result
            printf("Grid size: %d, Thread size: %d\n", GRIDS[i], THREADS);
            printf("GPU time used: %f seconds for sum = %d\n",
                   gpu_time_used / 1000.0f, final_host_data[0]);

            fprintf(fp, "%d,%d,%d,%f,%f\n", ARRAY_SIZE, GRIDS[i], THREADS,
                    cpu_time_used, gpu_time_used / 1000.0f);
        }

        // Free the memory
        free(initial_host_data);
        free(final_host_data);
        cudaFree(initial_device_data);
        cudaFree(final_device_data);
    }

    fclose(fp);

    return 0;
}

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

data = np.genfromtxt('output.csv', delimiter=',', skip_header=1)
data = data[data[:, 0].argsort()]
dataset_sizes = np.unique(data[:, 0])

for dataset_size in dataset_sizes:

    data_subset = data[data[:, 0] == dataset_size]
    grid_size = data_subset[:, 1]
    thread_size = data_subset[:, 2]
    gpu_time = data_subset[:, 4]
    cpu_time = data_subset[:, 3]

    plt.axhline(y=cpu_time[0], color='r', linestyle='-')
    plt.text(0, cpu_time[0], 'CPU time: ' + str(cpu_time[0]))
    plt.bar(range(3), gpu_time, width=0.5, align='center')
    plt.title('Dataset size: ' + str(dataset_size))
    plt.xlabel('<<<Grid size, Thread size>>>')
    plt.ylabel('GPU time')
    plt.xticks(range(3), zip(grid_size, thread_size))
    plt.show()